<a href="https://colab.research.google.com/github/danielegrattarola/ml-18-19/blob/master/02_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

Prof. Cesare Alippi  
Daniele Grattarola  ([`daniele.grattarola@usi.ch`](mailto:daniele.grattarola@usi.ch)  )    
Daniele Zambon ([`daniele.zambon@usi.ch`](mailto:daniele.zambon@usi.ch)  )

---
# Lab 02: Classification and neural networks

We have a $d$-dimensional input vector $X \in \mathbb{R}^d$ and a set of $k$ possible classes, $C_1, \dots, C_k$.
Our goal in classification is to assign $X$ to the correct class. 

In particular, our goal is to determine a __discriminant__ function to parition the input space.

![alt text](http://atriplex.info/blog/wp-content/uploads/2017/05/th_lda.png)

---
# Simple approach: linear regression

We can use the tools we already have to build a simple classifier. 

Given a problem with two classes $C_0, C_1$ we assign:

$$
y =
\begin{cases}
    0 \textrm{, if } X \textrm{ is of class } C_0 \\
    1 \textrm{, if } X \textrm{ is of class } C_1 \\
\end{cases}
$$

and we build a dataset of observations $\{(X_1, y_1), \dots, (X_n, y_n)\}$.

We then solve our linear model $y = X^T \theta$ as:

$$
\hat\theta = (X^T X)^{-1} X^T Y
$$

and predict new samples as:

$$
\hat y = 
\begin{cases}
0 \textrm{, if } X^T \hat\theta < 0.5 \\
1 \textrm{, if } X^T \hat\theta \ge 0.5
\end{cases}
$$

We call __decision boundary__ the point that partitions the input space where $X^T \hat\theta = 0.5$.

Let's see this in code...

In [0]:
# Import some libraries and functions
import numpy as np
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt

# Create a classification problem
x, y = make_classification(n_features=1,           # Dimension of the input x
                           n_informative=1,        # Number of features that actually have meaning
                           n_clusters_per_class=1, # Number of clusters in a class
                           n_redundant=0,          # Number of features that carry the same information 
                           class_sep=2)            # Separation coefficient of the two classes

# Let's look at the data
plt.scatter(x, np.zeros_like(x))
plt.xlabel('X')
plt.yticks([]);

In [0]:
# Let's see the data as we would in a regression problem
plt.scatter(x, y)
plt.xlabel('X')
plt.ylabel('y');

In [0]:
# Now estimate the regression parameters from the data
X = np.hstack((x, np.ones((x.shape[0], 1))))  # Add dummy column for bias
S = np.dot(X.T, X)
S_inv = np.linalg.inv(S) 
theta_hat = S_inv.dot(X.T).dot(y)

print('Estimated theta: {}'.format(theta_hat))

# Plot the estimated linear model
y_hat = X.dot(theta_hat)
plt.plot(X[:, 0], y_hat)
plt.scatter(X[:, 0], y)
plt.ylabel('y')
plt.xlabel('X');

Are we happy with the result? We can plot the __decision boundary__ of our linear regression by imposing that 

$$
x_0 \cdot \hat\theta_0 + \hat\theta_1 = 0.5,
$$

which gives us: 

$$
x_0 = \frac{(0.5 - \hat\theta_1)}{\hat\theta_0}
$$

In [0]:
# Redo plots above
plt.plot(x, y_hat)
plt.scatter(x, y)
plt.ylabel('y')
plt.xlabel('X')

# Plot the decision boundary
decision_boundary = (0.5 - theta_hat[1]) / theta_hat[0]
plt.axvline(decision_boundary, c='g');

# Perceptron

The perceptron is a classifier introduced in 1957 by the U.S. Navy. 

When it was presented, the [NY Times](https://www.nytimes.com/1958/07/08/archives/new-navy-device-learns-by-doing-psychologist-shows-embryo-of.html) [reported](https://sci-hub.tw/10.1177/030631296026003005) that the perceptron was _"the embryo of an electronic computer that [...] will be able to walk, talk, see, write, reproduce itself and be conscious of its existence."_

Today, stacks of perceptrons are used to [walk](https://openai.com/blog/openai-baselines-ppo/), [talk](https://deepmind.com/blog/wavenet-generative-model-raw-audio/),  [see](https://arxiv.org/abs/1409.1556), [write](https://openai.com/blog/better-language-models/), and [reproduce themselves](https://github.com/floodsung/Meta-Learning-Papers), but self-consciousness is still a bit far...

--- 

Conceptually, the perceptron is a simple linear model followed by a Heaviside __activation function__: 

$$
f(X; W) = \textrm{heaviside}(\sum\limits_{i=1}^{m}x_i w_i) =  \textrm{heaviside}(X^TW)
$$

with the difference that the training is performed iteratively as: 

$$
W^{(i+1)} = W^{(i)} + \alpha \cdot (y - f(X, W^{(i)}) ) \cdot X
$$

$\alpha$ is called the __learning rate__, and it is a way of telling the model by how much to update the parameters $w$.

Note that, because of its simple training algorithm, the perceptron is only guaranteed to converge on __linearly separable__ problems. 

Let's see it in action on a problem with 2D features...




In [0]:
x, y = make_classification(n_features=2,           # Dimension of the input x
                           n_informative=2,        # Number of features that actually have meaning
                           n_clusters_per_class=1, # Number of clusters in a class
                           n_redundant=0,          # Number of features that carry the same information
                           class_sep=3)            # Separation coefficient of the two classes

# Let's look at the data
c = ['b' if _ == 0 else 'r' for _ in y]  # Tells matplotlib to assign blue to class "0" or red to class "1"
plt.scatter(x[:, 0], x[:, 1], color=c)
plt.xlabel('X_0')
plt.ylabel('X_1');

In [0]:
# Building a perceptron
X = np.hstack((x, np.ones((x.shape[0], 1))))  # Prep the inputs
w_init = np.random.normal(0, 1, size=(3,))    # Random weights (with bias)
w = w_init

# Activation / decision function
def heaviside(a):
    return 0 if a < 0 else 1

In the case of linear regression, we manually set the decision boundary s.t. $X^T \hat\theta = 0.5$.

For the perceptron, we do something similar, but the boundary is defined by the Heaviside step function

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Dirac_distribution_CDF.svg/325px-Dirac_distribution_CDF.svg.png)

so that we we predict the class of a sample as:

$$
\hat y = 
\begin{cases}
0 \textrm{, if } X^T W < 0 \\
1 \textrm{, if } X^T W \ge 0
\end{cases}
$$

The decision boundary line, then, is obtained as before, by imposing: 

$$
X^T W = 0.
$$

In our 2D case, this becomes: 

$$
x_0 \cdot w_0 +x_1 \cdot w_1 + w_2 = 0
$$

which gives us a line: 

$$
x_1 = \frac{-x_0 \cdot w_0 - w_2}{w_1} = - \frac{w_0}{w_1} \cdot x_0 - \frac{w_2}{w_1}
$$



In [0]:
# Plots
plt.scatter(X[:, 0], X[:, 1], color=c)
plt.xlabel('X_0')
plt.ylabel('X_1')

# Plot decision boundary
decision_boundary = - (w[0] / w[1]) * X[:, 0] - (w[2] / w[1])
plt.plot(X[:, 0], decision_boundary, c='g');

In [0]:
# Let's train the perceptron
learning_rate = 0.001 
for _ in range(10):
    for s in range(X.shape[0]):
        activation = X[s].dot(w)               # Compute activation
        prediction = heaviside(activation)     # Make a prediction
        error = y[s] - prediction              # Compute the prediction error
        update = learning_rate * error * X[s]  # Compute the update
        w = w + update                         # Update parameters
    print('New w: {}'.format(w))
    
plt.scatter(X[:, 0], X[:, 1], color=c)
plt.xlabel('X_1')
plt.ylabel('X_2');

# Plot decision boundry
decision_boundary = - (w[0] / w[1]) * X[:, 0] - (w[2] / w[1])
plt.plot(x[:, 0], decision_boundary, c='g');

In [0]:
from matplotlib import rc
from matplotlib import animation
rc('animation', html='jshtml')
rc('animation', embed_limit=400)

# Reset weights
w = w_init

# Train the perceptron
weights = []  # We will save weights here
for _ in range(10):
    for s in range(X.shape[0]):
        activation = X[s].dot(w)
        prediction = heaviside(activation)
        error = y[s] - prediction
        update = learning_rate * error * X[s]
        w = w + update
        weights.append(w)

# Animation
fig, ax = plt.subplots()
scatter = plt.scatter(X[:, 0], X[:, 1], color=c)
ax = plt.axis([X[:, 0].min(), X[:, 0].max(), X[:, 1].min(), X[:, 1].max()])

# Setup animation
w = weights[0]
decision_boundary = - (w[0] / w[1]) * X[:, 0] - (w[2] / w[1])
line, = plt.plot(X[:, 0], decision_boundary, c='g')

def animate(i):
    w = weights[i]
    decision_boundary = - (w[0] / w[1]) * X[:, 0] - (w[2] / w[1])
    line.set_data(X[:, 0], decision_boundary)
    return line,

# Create animation using the animate() function
anim = animation.FuncAnimation(fig, animate, frames=len(weights), interval=10, blit=True, repeat=True)
anim

# Feed-forward neural newtorks


Also known as __multi-layer perceptrons__ , neural networks are computational models inspired by the connected structure of the brain. The core component of neural networks is the neuron, which is composed of a perceptron and an activation function: 

$$
f(X; W) = \sigma ( X^T W).
$$

The main idea behind neural networks is to compose neurons in two different ways: 

1. by taking many neurons __in parallel__;
2. by composing many subsequent __layers__ of neurons;

The result is a network of neurons that take data as input, and compute sequential transformations until the desired result is produced as output.

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Colored_neural_network.svg/300px-Colored_neural_network.svg.png)

---

Take the neural network pictured above. Each of the four neurons labeled "Hidden", has a very similar form to the perceptron we already saw. 

In fact, we can write the four neurons as a system: 

$$
\begin{cases}
h_0 = \sigma(X^T w_0)  = \sigma(x_0w_{00} + x_1w_{01} + x_2w_{02}) \\
h_1 = \sigma(X^T w_1) = \sigma(x_0w_{10} + x_1w_{11} + x_2w_{12}) \\
h_2 = \sigma(X^T w_2) = \sigma(x_0w_{20} + x_1w_{21} + x_2w_{22}) \\
h_3 = \sigma(X^T w_3) =\sigma( x_0w_{30} + x_1w_{31} + x_2w_{32}) \\
\end{cases}
$$

or, better yet, as a vector: 

$$
\begin{bmatrix} 
h_0 \\
h_1 \\
h_2 \\
h_3 
\end{bmatrix}
=
\sigma\left(
\begin{bmatrix} 
x_0 \\
x_1 \\
x_2
\end{bmatrix}^{\textrm{ }T}
\begin{bmatrix} 
w_{00} & w_{10} & w_{20} & w_{30} \\
w_{01} & w_{11} & w_{21} & w_{31} \\
w_{02} & w_{12} & w_{22} & w_{32} \\
\end{bmatrix}
\right)^{\textrm{ }T}
$$

In short, we write the output of a __layer__ of neurons as:
$$
H = \sigma(X^TW)^T
$$

The same is true for the two "Output" neurons, with the difference that their input is not $X$, as for the hidden neurons, but it is the output $H$ of the hidden neurons. The output layer can be written as: 

$$
Y = \sigma(H^TV)^T
$$

(note that $V$ is a different matrix of parameters).

Finally, stacking the two layers simply means __composing__ them together, so that the whole neural network can be written as: 

$$
f(X; W, V) = \sigma(\sigma(X^T W) V)^T
$$

---
Neural networks are trained with a technique called __stochastic gradient descent__ (SGD) which is very similar to the perceptron update rule, only generalized to deal with many layers and parallel neurons. The key idea behind SGD is to update all the parameters of the network at the same time, based on how each parameter contributed to the __loss__ function $L(y, f(X; W))$.

Here, with $W$ we denote __all__ the parameters on the network, for brevity. 

The generalized update rule reads: 

$$
W^{(i+1)} = W^{(i)} + \alpha \frac{\partial L(y, f(X, W^{(i)}))}{\partial w^{(i)}}
$$

where $\alpha$ is again called __learning rate__.

---

When training neural networks for binary classification, we take the loss to be the __cross-entropy error function__: 

$$
L(W) =  -\frac1N \sum_{n=1}^N \bigg[y_n  \log \hat y_n + (1 - y_n)  \log (1 - \hat y_n)\bigg]
$$


# Wine quality dataset

Let's get started with a basic classification problem. 

We are given a set of wine reviews, with the following characteristics: 

In [0]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(url, delimiter=';')

data.head(10)

In [0]:
# Let's look at the distribution of the reviews
data['quality'].hist();

We can turn this into a binary classification problem by setting a threshold on the reviews: was the wine good (>= 6) or not?

Notice also how the value of the features are not commensurable with one another. For instance, the "total sulfur dioxide" can have values up to 100, while the "density" is necessarily limited to be <= 1. 

While this in principle is not a problem for our machine learning models, in practice it can lead to issues in the training procedure.

To standardize the data, we compute the following transformation: 

$$
X_{\textrm{standardized}} = \frac{X - \textrm{mean}(X)}{\textrm{std}(X)}
$$

In [0]:
# Extract features
X = data[data.columns[:-1]].values

# Normalize features
X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

# Extact targets
quality = data['quality'].values.astype(int)
y = quality >= 6
plt.hist(y);

# Shuffle data
p = np.random.permutation(data.shape[0])
X = X[p]
y = y[p]

# Neural networks in Python

To build our neural network we will use [TensorFlow](https://www.tensorflow.org/), one of the most popular deep learning libraries for Python (the other being [PyTorch](https://pytorch.org/)). 
TensorFlow provides a huge number of functions, like Numpy, that can be used to manipulate arrays, but offers two great advantages w.r.t. Numpy: 

1. the computation can be accelerated on GPU via the CUDA library;
2. the library implements __automatic differentiation__, meaning that the most analytically complex step of training, the computation of the gradient, is handled for you.

While TensorFlow is a very powerful library that offers a fine-coarsened control over what you build, for this course we will skip the low level details and instead use the official high-level API for TensorFlow: [Keras](https://keras.io).

# Introduction to TensorFlow/Keras

![alt text](https://www.tensorflow.org/images/tf_logo_social.png)

(most of what follows is adapted from [the introduction on the TensorFlow website](https://www.tensorflow.org/guide/low_level_intro))

The most important things to understand about building neural networks with Tensorflow, are the concepts of __tensor__ and __computational graph__. 

## Tensors
A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's __rank__ is its number of dimensions, while its __shape__ is a tuple of integers specifying the array's length along each dimension. 

Intuitively, tensors are the TensorFlow version of Numpy arrays. In fact, TF and Numpy are heavily intertwined and arrays can often be fed to TF models without modifications. 

## Computational graph
A computational graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

- `tf.Operation` (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
- `tf.Tensor`: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.


For this exercise session (and most of the course, probably), you will only need to keep in mind one thing: when using TF, __you first define the computation, and then you provide the data__. 

This means that all of your operations will be defined on symbolic objects, and only at the end you will actually run the computation.

Don't worry if you don't get this at first, it will become clearer by doing. 

# Keras

![alt text](https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png)



Keras offers collections of TF operations already arranged to implement neural networks with little to no effort. 
For instance, building a layer of 4 neurons like the one we saw above is as easy as calling `Dense(4)`. That's it. 

Moreover, Keras offers a high-level API for doing all the usual steps that we usually do when training a neural network, like training on some data, evaluating the performance, and predicting on unseen data. 

The core data structure of Keras is a model, a way to organize layers. The simplest type of model is the `Sequential` model, a linear stack of layers.

---

In this exercise we will see how to build and train a simple neural network from scratch, in Keras.

In [0]:
from keras.models import Sequential
from keras.layers import Dense

network = Sequential()
network.add(Dense(32, activation='relu', input_shape=X.shape[1:]))
network.add(Dense(1, activation='sigmoid'))

network.compile(optimizer='sgd', 
                loss='binary_crossentropy', 
                metrics=['acc'])
network.fit(X, y, epochs=100, validation_split=0.2)

# Pokémon dataset: unbalanced classes

Let's look at a problem where we have a lot of __unbalance__ between the two classes in our data. 
This will allow us to look at several things: 

1. How to deal with data normalization
2. How to compute __class weights__
3. How to evaluate performance on unbalanced problems

---

Pokémon are fictional creatures that are central to the Pokémon franchise.
Among them, Legendary Pokémon are a group of incredibly rare and often very powerful Pokémon, generally featured prominently in the legends and myths of the Pokémon world [[source]](https://bulbapedia.bulbagarden.net/wiki/Legendary_Pok%C3%A9mon).

The task that we will tackle in this exercise is simple: can we tell whether a Pokémon is legendary or not, by looking at its stats?

Let's start by getting the data and looking at it...

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

url = 'https://raw.githubusercontent.com/lgreski/pokemonData/master/Pokemon.csv'
data = pd.read_csv(url)

data.head(10)

Like we did before, we will need to standardize the data in order to have commensurable features.

However, here we face a problem that we didn't have before: we have substantially less samples of one class w.r.t. the other. This means that that our neural network is likely to ignore samples with $y=1$, because getting right the samples for which $y=0$ will lead to a lower error. 

Would you study for an exam question that was only asked once by the professor, in previous years? Or would you focus on the more common exercises that are more likely to be asked again? :)


In [0]:
# Extract features
X = data[['HP', 'Attack', 'Defense', 'SpecialAtk', 'SpecialDef', 'Speed']].values
X = (X - X.mean(0)) / X.std(0)
print(X)

# Extact targets
y = data['Legendary'].values
plt.hist(y);

To deal with the __class unbalance__ we will use a simple trick, that will allow our model to learn better. 

The trick consists in __re-weighting__ the loss function, so that the error on rare samples will count more than the error on common samples:

$$
L_{\textrm{reweighted}}(y, f(X; W)) =
\begin{cases}
\lambda_0 L(y, f(X; W))\textrm{, if } y=0 \\
\lambda_1 L(y, f(X; W))\textrm{, if } y=1
\end{cases}
$$

Ideally, $\lambda_0$ and $\lambda_1$should represent how rare the respective classes are in the dataset. 
A common way of computing the two values automatically is as: 

$$
\lambda_i = \frac{\textrm{# samples in dataset}}{\textrm{# classes}\cdot\textrm{# samples of class } i}
$$

In Keras (and also in Scikit-learn) we call these values `class_weight`.

Let's see how to compute them...

In [0]:
# Split train / test / validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train)

# Compute class weights
n_pokemons = X_train.shape[0]
n_legendaries = y_train.sum()
class_weights = {0: n_pokemons / (2 * (n_pokemons - n_legendaries)),
                 1: n_pokemons / (2 * n_legendaries)}

print('Training data: {} legendaries out of {} mons'.format(n_legendaries, n_pokemons))
print('Training data: class weights {}'.format(class_weights))

In order to train a neural network in Keras using class weights, we only need to apport some minor modifications to the previous model:

In [0]:
from keras.models import Sequential
from keras.layers import Dense

network = Sequential()
network.add(Dense(32, activation='relu', input_shape=X.shape[1:]))
network.add(Dense(1, activation='sigmoid'))

network.compile('sgd', 'binary_crossentropy', weighted_metrics=['acc'])

network.fit(X, y, epochs=10, validation_data=[X_val, y_val], class_weight=class_weights)

Finally, let's analyze the __test__ performance of our model:

In [0]:
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(y_true, y_pred, classes, cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Adapted from https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.grid(None)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title='Confusion matrix',
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, cm[i, j],
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

eval_results = network.evaluate(X_test, y_test, verbose=False)
print('Loss: {:.4f} - Acc: {:.2f}'.format(*eval_results))

y_pred = network.predict(X_test)
y_pred = np.round(y_pred)
plot_confusion_matrix(y_test, y_pred, classes=['normal', 'legendary']); 